# Azure Computer Vision API Demo

This notebook demonstrates how to use Azure Computer Vision REST APIs for image analysis and vectorization.

## Prerequisites

Set the following environment variables:
- `AZURE_CV_ENDPOINT` - Azure Computer Vision endpoint URL
- `AZURE_CV_KEY1` - Azure Computer Vision API key
- `AZURE_CV_REGION` - Azure region

## A note on sample images

The following resources are used in order to provide samples image inputs.

1. [PlaceCats](https://placecats.com) 
2. Public Domain images from Unsplash: [see this collection](https://unsplash.com/collections/N2A19eP1Ers/cv-samples)

## Setup: Import Libraries and Configure Environment

In [1]:
import os
import requests
import json
from typing import Dict, Any, List
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
from rich import print_json

# Load environment variables
AZURE_CV_ENDPOINT = os.getenv("AZURE_CV_ENDPOINT")
AZURE_CV_KEY = os.getenv("AZURE_CV_KEY1")
AZURE_CV_REGION = os.getenv("AZURE_CV_REGION")

# Verify configuration
if not all([AZURE_CV_ENDPOINT, AZURE_CV_KEY, AZURE_CV_REGION]):
    print(
        "⚠️ Warning: Please set AZURE_CV_ENDPOINT, AZURE_CV_KEY1, and AZURE_CV_REGION environment variables"
    )
else:
    print("✅ Configuration loaded successfully")

## Helper Functions

In [ ]:
def analyze_image(
    image_url: str, features: str, language: str = "en"
) -> Dict[str, Any]:
    """
    Analyze an image using Azure Computer Vision API.

    Args:
        image_url: URL of the image to analyze
        features: Comma-separated list of visual features (tags, caption, denseCaptions, objects, read, people)
        language: Language code (default: 'en')

    Returns:
        Dictionary containing the API response
    """
    url = f"{AZURE_CV_ENDPOINT}/computervision/imageanalysis:analyze"

    params = {"features": features, "language": language, "api-version": "2024-02-01"}

    headers = {
        "Ocp-Apim-Subscription-Key": AZURE_CV_KEY,
        "Content-Type": "application/json",
    }

    body = {"url": image_url}

    response = requests.post(url, params=params, headers=headers, json=body)
    response.raise_for_status()

    return response.json()


def print_response(response: Dict[str, Any]):
    """Pretty print API response."""
    print_json(json.dumps(response, indent=2))


def draw_bounding_boxes(
    image_url: str,
    people_result: Dict[str, Any],
    confidence_threshold: float = 0.5,
    box_color: str = "red",
    box_width: int = 3,
) -> Image.Image:
    """
    Draw bounding boxes on an image for detected people.

    Args:
        image_url: URL of the image
        people_result: The peopleResult data from the API response
        confidence_threshold: Minimum confidence to draw a box (default: 0.5)
        box_color: Color of the bounding box (default: 'red')
        box_width: Width of the bounding box line (default: 3)

    Returns:
        PIL Image with bounding boxes drawn
    """
    # Download the image
    response = requests.get(image_url)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))

    # Create a drawing context
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes for each detected person
    for person in people_result.get("values", []):
        confidence = person.get("confidence", 0)

        # Only draw boxes above confidence threshold
        if confidence >= confidence_threshold:
            bbox = person["boundingBox"]
            x, y, w, h = bbox["x"], bbox["y"], bbox["w"], bbox["h"]

            # Draw rectangle
            draw.rectangle([(x, y), (x + w, y + h)], outline=box_color, width=box_width)

            # Add confidence label
            label = f"{confidence:.2%}"
            # Draw label background
            text_bbox = draw.textbbox((x, y - 20), label)
            draw.rectangle(text_bbox, fill=box_color)
            draw.text((x, y - 20), label, fill="white")

    return img


def draw_object_bounding_boxes(
    image_url: str,
    objects_result: Dict[str, Any],
    confidence_threshold: float = 0.5,
    box_color: str = "blue",
    box_width: int = 3,
) -> Image.Image:
    """
    Draw bounding boxes on an image for detected objects.

    Args:
        image_url: URL of the image
        objects_result: The objectsResult data from the API response
        confidence_threshold: Minimum confidence to draw a box (default: 0.5)
        box_color: Color of the bounding box (default: 'blue')
        box_width: Width of the bounding box line (default: 3)

    Returns:
        PIL Image with bounding boxes drawn
    """
    # Download the image
    response = requests.get(image_url)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))

    # Create a drawing context
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes for each detected object
    for obj in objects_result.get("values", []):
        # Get the highest confidence tag
        tags = obj.get("tags", [])
        if not tags:
            continue

        # Sort by confidence and get the top tag
        top_tag = max(tags, key=lambda t: t.get("confidence", 0))
        confidence = top_tag.get("confidence", 0)
        object_name = top_tag.get("name", "Unknown")

        # Only draw boxes above confidence threshold
        if confidence >= confidence_threshold:
            bbox = obj["boundingBox"]
            x, y, w, h = bbox["x"], bbox["y"], bbox["w"], bbox["h"]

            # Draw rectangle
            draw.rectangle([(x, y), (x + w, y + h)], outline=box_color, width=box_width)

            # Add label with object name and confidence
            label = f"{object_name} ({confidence:.1%})"
            # Draw label background
            text_bbox = draw.textbbox((x, y - 25), label)
            draw.rectangle(text_bbox, fill=box_color)
            draw.text((x, y - 25), label, fill="white")

    return img


def draw_text_bounding_boxes(
    image_url: str,
    read_result: Dict[str, Any],
    box_color: str = "green",
    box_width: int = 2,
    show_text: bool = True,
) -> Image.Image:
    """
    Draw bounding boxes on an image for detected text (OCR).

    Args:
        image_url: URL of the image
        read_result: The readResult data from the API response
        box_color: Color of the bounding box (default: 'green')
        box_width: Width of the bounding box line (default: 2)
        show_text: Whether to show text labels (default: True)

    Returns:
        PIL Image with bounding boxes drawn
    """
    # Download the image
    response = requests.get(image_url)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))

    # Create a drawing context
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes for each line of text
    for block in read_result.get("blocks", []):
        for line in block.get("lines", []):
            text = line.get("text", "")
            bounding_polygon = line.get("boundingPolygon", [])

            if len(bounding_polygon) >= 4:
                # Convert polygon points to tuple list
                points = [(p["x"], p["y"]) for p in bounding_polygon]

                # Draw polygon (handles rotated text better than rectangles)
                draw.polygon(points, outline=box_color, width=box_width)

                # Optionally add text label
                if show_text and text:
                    # Get the top-left corner for label placement
                    x, y = points[0]

                    # Draw text background
                    text_bbox = draw.textbbox((x, y - 20), text)
                    draw.rectangle(text_bbox, fill=box_color)
                    draw.text((x, y - 20), text, fill="white")

    return img


def display_image(image_url: str) -> Image.Image:
    """
    Download and display an image from a URL.

    Args:
        image_url: URL of the image to display

    Returns:
        PIL Image object
    """
    response = requests.get(image_url)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))
    return img


def draw_dense_captions(
    image_url: str,
    dense_captions_result: Dict[str, Any],
    box_color: str = "purple",
    box_width: int = 3,
) -> Image.Image:
    """
    Draw bounding boxes on an image for dense captions.

    Args:
        image_url: URL of the image
        dense_captions_result: The denseCaptionsResult data from the API response
        box_color: Color of the bounding box (default: 'purple')
        box_width: Width of the bounding box line (default: 3)

    Returns:
        PIL Image with bounding boxes and captions drawn
    """
    # Download the image
    response = requests.get(image_url)
    response.raise_for_status()
    img = Image.open(BytesIO(response.content))

    # Create a drawing context
    draw = ImageDraw.Draw(img)

    # Draw bounding boxes for each dense caption
    for idx, caption_obj in enumerate(dense_captions_result.get("values", [])):
        text = caption_obj.get("text", "")
        confidence = caption_obj.get("confidence", 0)
        bbox = caption_obj.get("boundingBox", {})

        x, y, w, h = (
            bbox.get("x", 0),
            bbox.get("y", 0),
            bbox.get("w", 0),
            bbox.get("h", 0),
        )

        # Draw rectangle
        draw.rectangle([(x, y), (x + w, y + h)], outline=box_color, width=box_width)

        # Add caption text and confidence
        label = f"{text} ({confidence:.1%})"

        # Draw label background
        text_bbox = draw.textbbox((x, y - 25), label)
        draw.rectangle(text_bbox, fill=box_color)
        draw.text((x, y - 25), label, fill="white")

    return img

## 1. Analyze Image: Tags

Extract content tags from the image.

In [ ]:
image_url = "https://placecats.com/300/200"
response = analyze_image(image_url, features="tags", language="en")
print_response(response)
display_image(image_url)

## 2. Analyze Image: Caption

Generate a single caption describing the image.

In [5]:
image_url = "https://placecats.com/louie/300/200"
response = analyze_image(image_url, features="caption", language="en")
print_response(response)
display_image(image_url)

## 3. Analyze Image: Dense Captions

Generate detailed captions for different regions of the image.

In [6]:
image_url = "https://placecats.com/millie_neo/300/200"
response = analyze_image(image_url, features="denseCaptions", language="en")
print_response(response)
display_image(image_url)

### Visualize Dense Captions with Bounding Boxes

Dense captions provide detailed descriptions for different regions of the image. Let's visualize where each caption applies.

In [ ]:
# Draw bounding boxes with captions overlaid
img_with_captions = draw_dense_captions(image_url, response["denseCaptionsResult"])

# Display the image
img_with_captions

## 4. Analyze Image: Objects

Detect and locate objects in the image.

In [ ]:
# drink
# image_url = "https://images.unsplash.com/photo-1760662018512-e881f06d0fe0?q=80&w=1024"

# tools (😟)
# image_url = "https://images.unsplash.com/photo-1567361808960-dec9cb578182?q=80&w=1024"

# items in a purse
# image_url = "https://images.unsplash.com/photo-1602532360508-595f449c7c55?q=80&w=1024"

# desk
image_url = "https://images.unsplash.com/photo-1625461291092-13d0c45608b3?q=80&w=1024"

# produce
# image_url = "https://images.unsplash.com/photo-1635341083777-5f93a755e916?q=80&w=1024"

# Ramen
# image_url = "https://images.unsplash.com/photo-1504674900247-0877df9cc836?q=80&w=1024"

response = analyze_image(image_url, features="objects", language="en")
print_response(response)

### Visualize Object Detection with Bounding Boxes

Draw bounding boxes on the image to visualize detected objects. Each box is labeled with the object name and confidence.

In [29]:
# Draw bounding boxes for objects detected with confidence >= 50%
img_with_boxes = draw_object_bounding_boxes(
    image_url, response["objectsResult"], confidence_threshold=0.5
)

# Display the image
img_with_boxes

You can adjust parameters to customize the visualization:

In [27]:
# Show all detections with custom styling
img_all_objects = draw_object_bounding_boxes(
    image_url,
    response["objectsResult"],
    confidence_threshold=0.3,  # <--- Lower the threshold
    box_color="yellow",  # <--- Change the box color
    box_width=8,  # <--- Change the box size
)

img_all_objects

## 5. Analyze Image: Read (OCR)

Extract text from the image using Optical Character Recognition.

In [ ]:
# Text only
image_url = "https://images.unsplash.com/photo-1592496000931-e50d83df1286?q=80&w=1024"

# Placard
# image_url = "https://images.unsplash.com/photo-1495001258031-d1b407bc1776?q=80&w=1024"
response = analyze_image(image_url, features="read", language="en")
print_response(response)

### Visualize OCR Text Detection with Bounding Boxes

Draw bounding boxes around detected text. The function uses polygon shapes to accurately represent rotated or skewed text.

In [16]:
# Draw bounding boxes for detected text with labels
img_with_text_boxes = draw_text_bounding_boxes(
    image_url, response["readResult"], show_text=True
)

# Display the image
img_with_text_boxes

You can also show just the bounding boxes without text labels:

In [17]:
# Draw bounding boxes only (no text labels)
img_boxes_only = draw_text_bounding_boxes(
    image_url,
    response["readResult"],
    show_text=False,
    box_color="orange",
    box_width=3,
)

img_boxes_only

## 6. Analyze Image: People

Detect people in the image.

In [18]:
# women with some drinks
image_url = "https://images.unsplash.com/photo-1532635241-17e820acc59f?q=80&w=1024"

# woman on roof
# image_url = "https://images.unsplash.com/photo-1589156288859-f0cb0d82b065?q=80&w=1024"

# couple  on a bench
# image_url = "https://images.unsplash.com/photo-1508963493744-76fce69379c0?q=80&w=1024"

# people in a crosswalk
# image_url = "https://images.unsplash.com/photo-1548783307-f63adc3f200b?q=80&w=1024"

response = analyze_image(image_url, features="people", language="en")
print_response(response)

### Visualize People Detection with Bounding Boxes

Draw bounding boxes on the image to visualize detected people. By default, only shows detections with confidence >= 50%.

In [19]:
# Draw bounding boxes for people detected with confidence >= 50%
img_with_boxes = draw_bounding_boxes(
    image_url, response["peopleResult"], confidence_threshold=0.5
)

# Display the image
img_with_boxes

You can adjust the confidence threshold to see more or fewer detections:

In [20]:
# Show all detections (including low confidence)
img_all_detections = draw_bounding_boxes(
    image_url,
    response["peopleResult"],
    confidence_threshold=0.0,
    box_color="green",
    box_width=2,
)

img_all_detections

## 7. Vectorize Image

Generate vector embeddings for the image, useful for image similarity search and retrieval.

In [21]:
def vectorize_image(
    image_url: str, model_version: str = "2023-04-15"
) -> Dict[str, Any]:
    """
    Vectorize an image using Azure Computer Vision API.

    Args:
        image_url: URL of the image to vectorize
        model_version: Model version to use (default: '2023-04-15')

    Returns:
        Dictionary containing the API response with vector embeddings
    """
    url = f"{AZURE_CV_ENDPOINT}/computervision/retrieval:vectorizeImage"

    params = {"model-version": model_version, "api-version": "2024-02-01"}

    headers = {
        "Ocp-Apim-Subscription-Key": AZURE_CV_KEY,
        "Content-Type": "application/json",
    }

    body = {"url": image_url}

    response = requests.post(url, params=params, headers=headers, json=body)
    response.raise_for_status()

    return response.json()


# Vectorize the image (using our produce image)
image_url = "https://images.unsplash.com/photo-1635341083777-5f93a755e916?q=80&w=1024"
response = vectorize_image(image_url, model_version="2023-04-15")

# Print response (showing vector dimensions)
if "vector" in response:
    vector = response["vector"]
    print(f"Vector dimensions: {len(vector)}")
    print(f"First 10 values: {vector[:10]}")
    print(f"\nFull response structure:")
    print(
        json.dumps(
            {
                k: v if k != "vector" else f"<{len(v)} dimensional vector>"
                for k, v in response.items()
            },
            indent=2,
        )
    )
else:
    print_response(response)

## Example: Combine Multiple Features

You can analyze multiple features in a single API call by combining them with commas.

In [ ]:
# Analyze multiple features at once
# woman in shopping cart.
image_url = "https://images.unsplash.com/photo-1643443578657-6170bfbb1920?q=80&w=1024"
response = analyze_image(image_url, features="tags,caption,objects", language="en")
# print_response(response)

caption_confidence = round(response["captionResult"]["confidence"], 2)
caption = response["captionResult"]["text"]
tags = [
    f"{d["name"]} ({round(d["confidence"], 2)})"
    for d in response["tagsResult"]["values"]
]

print(f"CAPTION: {caption_confidence}: {caption}")
print(f"TAGS: {"\n".join(tags)}")

img = draw_object_bounding_boxes(
    image_url,
    response["objectsResult"],
    confidence_threshold=0.5,
    box_color="green",
    box_width=8,
)
img